In [1]:
import pandas as pd
import numpy as np
import csv
import MeCab
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
import re
import neologdn
import demoji
import emoji

# model_dir = '/Users/iomacbookair2/Documents/lab/DEIM2023/entity_vector/entity_vector.model.bin'
model_dir = '/Users/labimac/Documents/lab/DEIM2023/entity_vector/entity_vector.model.bin'
model_word2vec = KeyedVectors.load_word2vec_format(model_dir, binary=True)

In [2]:
def calc_similarity_word2vec(sentence1, sentence2, model):
  mecab = MeCab.Tagger(
      '-d /opt/homebrew/lib/mecab/dic/mecab-ipadic-neologd')

  pre_sentence1 = sentence1.split(" ")
  sentence1_words = []
  for s1 in pre_sentence1:
    sentence1_words += [line.split("\t")[0]
                        for line in mecab.parse(s1).split("\n")[:-2]]
  # print(f"sentence1_words: {sentence1_words}")

  pre_sentence2 = sentence2.split(" ")
  sentence2_words = []
  for s2 in pre_sentence2:
    sentence2_words += [line.split("\t")[0]
                        for line in mecab.parse(s2).split("\n")[:-2]]
  # print(f"sentence2_words: {sentence2_words}")

  if not sentence1_words and not sentence2_words:
    similarity = 1
  elif not sentence1_words or not sentence2_words:
    similarity = 0
  elif (len(sentence1_words) == 1 and len(sentence2_words) != 1) or (len(sentence2_words) == 1 and len(sentence1_words) != 1):
    similarity = 0
  elif all(re.match(r'.*[a-zA-Z].*', word) for word in sentence1_words) or all(re.match(r'.*[a-zA-Z].*', word) for word in sentence2_words):
    similarity = 0
  else:
    # Compute word embeddings for each sentence
    sentence1_embedding = np.mean([model[word]
                                  for word in sentence1_words if word in model], axis=0)
    # print(f"sentence1_embedding: {sentence1_embedding}")
    sentence2_embedding = np.mean([model[word]
                                  for word in sentence2_words if word in model], axis=0)
    # print(f"sentence2_embedding: {sentence2_embedding}")
    if np.isnan(sentence1_embedding).any() or np.isnan(sentence2_embedding).any():
        similarity = 0
    else:
      similarity = cosine_similarity(
          [sentence1_embedding], [sentence2_embedding])[0][0]

  # print(f"similarity: {similarity}")

  return similarity


def calc_average_similarity_word2vec(sentences, key):
    similarities = []
    for i in range(len(sentences)):
        for j in range(i + 1, len(sentences)):
            similarity = calc_similarity_word2vec(
                sentences[i], sentences[j], model_word2vec)
            similarities.append(similarity)

    q = [0, 0.25, 0.5, 0.75, 1]
    outputs = {}
    for i in range(len(q)):
        outputs[f"quantile_{i}"] = np.quantile(similarities, q[i])
    outputs["average_similarity"] = np.mean(similarities)
    outputs["size"] = len(list(np.sort(np.array(similarities))))
    outputs["width_q"] = outputs["quantile_3"] - outputs["quantile_1"]
    outputs["standard_deviation"] = np.std(similarities)

    if key == "quantile_3":
        return outputs["quantile_3"]
    elif key == "average":
        return outputs["average_similarity"]
    elif key == "size":
        return outputs["size"]
    elif key == "width":
        return outputs["width_q"]
    elif key == "standard_deviation":
        return outputs["standard_deviation"]


In [3]:
# path = "/Users/iomacbookair2/Documents/lab/DEIM2023/tweet_csv/221216_ann_fri.csv"
path = "/Users/labimac/Documents/lab/DEIM2023/tweet_csv/221216_ann_fri.csv"
df = pd.read_csv((path))
df.sort_values(by = 'created_at', ascending = True, inplace = True)
df = df.reset_index(drop=True)
df['created_at'] = pd.to_datetime(df['created_at'])
df = df.drop("author_id", axis=1)
df = df.drop("username", axis=1)

df.to_csv("sorted.csv", index=False)
df

,created_at,text
0,2022-12-17 01:00:00,#霜降り明星ANN
1,2022-12-17 01:00:00,#霜降り明星ANN
2,2022-12-17 01:00:00,#霜降り明星ANN
3,2022-12-17 01:00:00,#霜降り明星ANN
4,2022-12-17 01:00:00,#霜降り明星ANN
...,...,...
1744,2022-12-17 03:02:09,霜降り明星のおふたり、 スタッフのみなさん、 リスナーのみなさん、 お疲れさまでした🍀 来週...
1745,2022-12-17 03:02:51,楽しかった #霜降り明星ANN
1746,2022-12-17 03:03:02,「霜降り明星のオールナイトニッポン」12/16 ・収録放送でリスナーからの刺激的な質問に解答...
1747,2022-12-17 03:03:09,お疲れ様でした！！ 今週も面白かったです♪ #霜降り明星ANN


In [4]:
def preprocess(text):
    text = emoji.replace_emoji(text, replace=' ')
    text = neologdn.normalize(text)
    text = re.sub(r'#\S+', '', text) # ハッシュタグをスペースに置き換え
    text = re.sub(r'＃\S+', '', text) # ハッシュタグをスペースに置き換え
    text = re.sub(r'http?://[\w/:%#\$&\?\(\)~\.=\+\-]+', '', text) # URLをスペースに置き換え
    text = re.sub(r'https?://\S+', ' ', text) # URLをスペースに置き換え
    text = re.sub(r'[!-/:-@[-`{-~]', r' ', text) # 記号をスペースに置き換え
    text = re.sub(
        u'[■-♯【】「」『』・ㅂﾟˊᗜ、。∀〇╰ˋω…╭´｀•˘д↑艸╯→°д̀ᴗ˃˂⁽⁾φl└＼※彡𖥦←ꂹ]', ' ', text) # 記号をスペースに置き換え
    text = re.sub(r'(\d)([,.])(\d+)', r'\1\3', text) # 小数点とカンマを消す
    text = re.sub(r'\d+', '0', text) # 数字を0に置き換え
    text = text.lower() # 英字を小文字に
    text = re.sub(r"[\u3000\t\r\n]", " ", text) # 空白文字をスペースに置き換え
    return text


df['text'] = df['text'].apply(preprocess)
df
df.to_csv('filtered3.csv', index=False)


In [5]:
groups = df.groupby(pd.Grouper(key='created_at', freq='min'))
df_texts_by_minute = pd.DataFrame({
    "texts_by_minute": groups.apply(lambda x: x["text"].tolist())
})
df_texts_by_minute = df_texts_by_minute.reset_index()
df_texts_by_minute
# df_texts_by_minute.to_csv("texts_by_minute.csv", index=False)

,created_at,texts_by_minute
0,2022-12-17 01:00:00,"[, , , , , , , , , , , , , , , , , , , , , , ,..."
1,2022-12-17 01:01:00,"[, , , , , , , , いつもよりメール多かった, 今日録音だった, , , , ..."
2,2022-12-17 01:02:00,"[久しぶりにリアタイ , , トゥルーさん, ねむぅい, , , , , , ..."
3,2022-12-17 01:03:00,"[トゥルーマンさん , はたけさんお休み, , 高橋さん わりと喋る方だと思いますけど笑..."
4,2022-12-17 01:04:00,"[トゥルさんフワちゃんの時は喋るけどね, , 粗品さん フワは人を選んでいる せいやさん さ..."
...,...,...
119,2022-12-17 02:59:00,"[来週はフースーヤあざしたー, だした, 録音回でも安定の面白さお疲れ様でした だしてぃー,..."
120,2022-12-17 03:00:00,"[ばいちょーんおつかれさまでした, xxxお疲れ様でした , だしたー, お疲れ様でし..."
121,2022-12-17 03:01:00,[0 0くらい眠すぎてリアタイ無理やと思ってたけど結局今日も聴けてしまった 霜降りannすご...
122,2022-12-17 03:02:00,[霜降り明星のおふたり スタッフのみなさん リスナーのみなさん お疲れさまでした来週フースー...


In [6]:
df_texts_by_minute["median"] = df_texts_by_minute["texts_by_minute"].apply(
    lambda x: 0 if len(x) <= 1 else calc_average_similarity_word2vec(x, "quantile_3"))


/Users/labimac/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/labimac/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/labimac/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/labimac/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/labimac/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/labimac/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric

In [7]:
df_texts_by_minute["average"] = df_texts_by_minute["texts_by_minute"].apply(
    lambda x: 0 if len(x) <= 1 else calc_average_similarity_word2vec(x, "average"))


/Users/labimac/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/labimac/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/labimac/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/labimac/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/labimac/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/labimac/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric

In [8]:
df_texts_by_minute["size"] = df_texts_by_minute["texts_by_minute"].apply(
    lambda x: 0 if len(x) <= 1 else calc_average_similarity_word2vec(x, "size"))


/Users/labimac/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/labimac/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/labimac/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/labimac/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/labimac/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/labimac/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric

In [9]:
df_texts_by_minute["width"] = df_texts_by_minute["texts_by_minute"].apply(
    lambda x: 0 if len(x) <= 1 else calc_average_similarity_word2vec(x, "width"))


/Users/labimac/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/labimac/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/labimac/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/labimac/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/labimac/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/labimac/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric

In [10]:
df_texts_by_minute["standard_deviation"] = df_texts_by_minute["texts_by_minute"].apply(
    lambda x: 0 if len(x) <= 1 else calc_average_similarity_word2vec(x, "standard_deviation"))


/Users/labimac/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/labimac/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/labimac/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/labimac/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/labimac/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/labimac/miniforge3/envs/lab/lib/python3.9/site-packages/numpy/core/fromnumeric

In [11]:
df_texts_by_minute

,created_at,texts_by_minute,median,average,size,width,standard_deviation
0,2022-12-17 01:00:00,"[, , , , , , , , , , , , , , , , , , , , , , ,...",1.000000,0.867152,20100,0.000000,0.338145
1,2022-12-17 01:01:00,"[, , , , , , , , いつもよりメール多かった, 今日録音だった, , , , ...",1.000000,0.562441,1485,1.000000,0.480177
2,2022-12-17 01:02:00,"[久しぶりにリアタイ , , トゥルーさん, ねむぅい, , , , , , ...",0.494496,0.276390,630,0.494496,0.366344
3,2022-12-17 01:03:00,"[トゥルーマンさん , はたけさんお休み, , 高橋さん わりと喋る方だと思いますけど笑...",0.567447,0.326528,528,0.567447,0.317241
4,2022-12-17 01:04:00,"[トゥルさんフワちゃんの時は喋るけどね, , 粗品さん フワは人を選んでいる せいやさん さ...",0.548706,0.313005,210,0.548706,0.291965
...,...,...,...,...,...,...,...
119,2022-12-17 02:59:00,"[来週はフースーヤあざしたー, だした, 録音回でも安定の面白さお疲れ様でした だしてぃー,...",0.746862,0.563344,946,0.388081,0.249035
120,2022-12-17 03:00:00,"[ばいちょーんおつかれさまでした, xxxお疲れ様でした , だしたー, お疲れ様でし...",0.742405,0.515697,78,0.742405,0.337105
121,2022-12-17 03:01:00,[0 0くらい眠すぎてリアタイ無理やと思ってたけど結局今日も聴けてしまった 霜降りannすご...,0.770879,0.674326,36,0.192850,0.149347
122,2022-12-17 03:02:00,[霜降り明星のおふたり スタッフのみなさん リスナーのみなさん お疲れさまでした来週フースー...,0.645642,0.645642,1,0.000000,0.000000


In [12]:
df_texts_by_minute.to_csv('cosine_similarity_w2v_221216_ann_fri.csv', index=False)